In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import time
import deepdish as dd
from sequence_annotation.genome_handler.seq_container import AnnSeqContainer
from sequence_annotation.data_handler.fasta import read_fasta
from sequence_annotation.genome_handler.utils import loader
from sequence_annotation.pytorch.AS_space_finder import ASModelSpaceFinder
from sequence_annotation.pytorch.AS_model_facade import ASModelFacade
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import torch
torch.backends.cudnn.benchmark = True

Using TensorFlow backend.


In [3]:
h5=dd.io.load('../io/Arabidopsis_thaliana/data/2019_02_20/result/result_dist_to_five_1000_dist_to_three_1000_merged.h5')
fasta = read_fasta('../io/Arabidopsis_thaliana/data/2019_02_20/result/result_dist_to_five_1000_dist_to_three_1000_merged.fasta')
ann_seqs = AnnSeqContainer().from_dict(h5)

In [4]:
simplify_map={'exon':['exon'],'intron':['intron','mix'],'other':['other']}
selected_fasta,selected_seqs,o_f,o_s = loader(fasta,ann_seqs,simplify_map=simplify_map,ratio=1,
                                          min_len=0,max_len=4000,num_max=int(len(ann_seqs)/100),
                                             outlier_coef=1.2)

/home/sequence_annotation/sequence_annotation/genome_handler/ann_seq_processor.py:218: RuntimeWarning: invalid value encountered in true_divide
  vecs.append(np.nan_to_num(temp/temp))


In [40]:
trials = Trials()
facade = ASModelFacade(selected_fasta,selected_seqs,'../io/record/arabidopsis_2019_03_17',
                       ratio=[0.7,0.2,0.1],outlier_fasta=o_f,outlier_seq=o_s)

facade.patient=4
facade.batch_size=16
facade.epoch_num=100
facade.affect_length_max = 10000
facade.early_stop_target = 'val_loss'
facade.target_max = False

20 {'exon': 18041.0, 'intron': 4961.0, 'other': 40000.0}
6 {'exon': 5972.0, 'intron': 2596.0, 'other': 12000.0}
2 {'exon': 1272.0, 'intron': 1242.0, 'other': 4000.0}


In [97]:
facade.lr=1e-2

In [102]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.nn.init import ones_,zeros_,uniform_,normal_,constant_,eye_
from torch.nn import Hardtanh, Sigmoid,Tanh
from torch import randn
import numpy as np
from sequence_annotation.pytorch.customize_layer import noisy_hard_sigmoid,noisy_hard_tanh,noisy_relu,SeqAnnModel

def init_uniform_by_size(param):
    bound = (1/(param.shape[1]**0.5))
    return uniform_(param,-bound,bound)

class GatedIndRnnCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self._gate_num = 1
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.weights_i = nn.Parameter(torch.empty((hidden_size,input_size)))
        self.weights_s = nn.Parameter(torch.empty((hidden_size,hidden_size)))
        self.gate_i_bias = nn.Parameter(torch.empty(hidden_size))
        self.gate_f_bias = nn.Parameter(torch.empty(hidden_size))
        self.input_bias = nn.Parameter(torch.empty(hidden_size))
        self.pre_state_bias = nn.Parameter(torch.empty(hidden_size))
        self.gate_function =  noisy_hard_sigmoid(c=0.05)
        self.recurrent_function = noisy_relu(c=1)
        self.gate_i_weights = nn.Parameter(torch.empty(hidden_size,hidden_size+hidden_size))
        self.gate_f_weights = nn.Parameter(torch.empty((hidden_size,hidden_size+hidden_size)))
        self.ln = nn.LayerNorm(hidden_size)
        self.output_names = ['new_h','pre_gate_f','gate_f','values_i','pre_h']
        self.reset_parameters()

    def reset_parameters(self):
        init_uniform_by_size(self.gate_f_weights)
        init_uniform_by_size(self.gate_i_weights)
        init_uniform_by_size(self.weights_i)
        init_uniform_by_size(self.weights_s)
        constant_(self.gate_i_bias,1)
        constant_(self.gate_f_bias,1)
        constant_(self.input_bias,0)
        constant_(self.pre_state_bias,0)
    def forward(self, input, state):
        values_i = F.linear(input, self.weights_i,self.input_bias)
        #values_state = F.linear(, self.weights_s,self.pre_state_bias).tanh()
        cat = torch.cat([self.ln(state),values_i],1)
        pre_gate_i = F.linear(cat, self.gate_i_weights,self.gate_i_bias)
        pre_gate_f = F.linear(cat, self.gate_f_weights,self.gate_f_bias)
        gate_f = self.gate_function(pre_gate_f,training=self.training)
        gate_i = self.gate_function(pre_gate_i,training=self.training)
        pre_h = state*gate_f+ (values_i)*gate_i
        new_h = self.recurrent_function(pre_h,self.training)
        return new_h,pre_gate_f,gate_f,values_i,pre_h

In [103]:
space={
    'cnns_setting':{
        'kernel_size':60,
        "ln_mode":"before_cnn",
        'layer_num':0,
        'out_channels':16,
        'with_pwm':False
    },
    'rnns_setting':{
        'layer_num':2,
        'hidden_sizes':4,
        'tanh_after_rnn':True,
        'rnn_cell_class':GatedIndRnnCell,
        'init_value':0,
        'cnn_before_rnn':False
    },
    'pwm_before_rnns':False,
    'last_kenel_size':2
}

In [ ]:
for key,val in space.items():
    if key == 'kernel_size':
        space[key]=int(val)
cnn_num = space['cnns_setting']['layer_num']
cnn_kernel_size = space['cnns_setting']['kernel_size']
rnn_num = space['rnns_setting']['layer_num']
space['rnns_setting']['hidden_sizes']=[space['rnns_setting']['hidden_sizes']]*rnn_num
space['cnns_setting']['out_channels']=[space['cnns_setting']['out_channels']]*cnn_num
space['cnns_setting']['kernel_sizes']=[cnn_kernel_size]*cnn_num
model = SeqAnnModel(in_channels=4,out_channels=len(facade.ann_types),init_value=1,**space).cuda()
train_record = facade.train("model_gate_fi_post_relu_with_tanh_only_input",model)

Processing data...
Processing model..
Compiling model...
Processing worker...
Executing...
Start working(2019-03-19 10:16:23)


In [23]:
#trainer = ASModelSpaceFinder(facade,'../io/record/arabidopsis_2019_03_17')
#trainer.eval_target = 'val_loss'
#trainer.target_max = False
#best = fmin(trainer.objective, space, algo=tpe.suggest, max_evals=16,trials=trials)

In [29]:
import deepdish as dd
dd.io.save(trainer.root+'/records.h5',trainer.records)
dd.io.save(trainer.root+'/space_result.h5',trainer.space_result)